In [40]:
import pandas as pd
from fastai.tabular import * 
import sklearn.metrics as metrics
import numpy as np

In [42]:
df = pd.read_csv('/home/sharif/Data-Mining-Cup/train.csv', sep="|")

In [43]:
df.head()

,trustLevel,totalScanTimeInSeconds,grandTotal,lineItemVoids,scansWithoutRegistration,quantityModifications,scannedLineItemsPerSecond,valuePerSecond,lineItemVoidsPerPosition,fraud
0,5,1054,54.70,7,0,3,0.027514,0.051898,0.241379,0
1,3,108,27.36,5,2,4,0.129630,0.253333,0.357143,0
2,3,1516,62.16,3,10,5,0.008575,0.041003,0.230769,0
3,6,1791,92.31,8,4,4,0.016192,0.051541,0.275862,0
4,5,430,81.53,3,7,2,0.062791,0.189605,0.111111,0


In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1879 entries, 0 to 1878
Data columns (total 10 columns):
trustLevel                   1879 non-null int64
totalScanTimeInSeconds       1879 non-null int64
grandTotal                   1879 non-null float64
lineItemVoids                1879 non-null int64
scansWithoutRegistration     1879 non-null int64
quantityModifications        1879 non-null int64
scannedLineItemsPerSecond    1879 non-null float64
valuePerSecond               1879 non-null float64
lineItemVoidsPerPosition     1879 non-null float64
fraud                        1879 non-null int64
dtypes: float64(4), int64(6)
memory usage: 146.9 KB


In [45]:
dep = 'fraud'

In [46]:
cat = ['trustLevel']

In [47]:
cont = ['totalScanTimeInSeconds',
        'grandTotal',
        'lineItemVoids',
        'scansWithoutRegistration',
        'quantityModifications',
        'scannedLineItemsPerSecond',
        'valuePerSecond',
        'lineItemVoidsPerPosition'
       ]

In [48]:
procs = [Categorify, Normalize]

In [49]:
valid_idx = range(len(df)-300, len(df))
valid_idx

range(1579, 1879)

In [50]:
data = TabularDataBunch.from_df('.', df, dep, valid_idx=valid_idx, procs=procs, cat_names=cat)
print(data.train_ds.cont_names)
print(data.train_ds.cat_names)

['lineItemVoidsPerPosition', 'scansWithoutRegistration', 'valuePerSecond', 'lineItemVoids', 'totalScanTimeInSeconds', 'grandTotal', 'scannedLineItemsPerSecond', 'quantityModifications']
['trustLevel']


In [52]:
learn = tabular_learner(data, layers=[200,100], metrics=accuracy)
learn.fit_one_cycle(6, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.685336,0.562543,0.896667,00:00
1,0.433882,0.242663,0.913333,00:00
2,0.266030,0.158894,0.940000,00:00
3,0.186348,0.084197,0.970000,00:00
4,0.139351,0.086643,0.953333,00:00
5,0.105610,0.079336,0.966667,00:00


In [53]:
def loss(y_true, y_pred):
    tn, fp, fn, tp = metrics.confusion_matrix(y_true, y_pred).ravel()
    return (tn * 0 + fp * (-25) + fn * (-5) + tp * 5) / (tn + fp + fn + tp)

In [54]:
y = df['fraud'].iloc[1579:].to_numpy()
y

array([0, 0, 0, 0, ..., 1, 0, 0, 0])

In [55]:
y_hat = [int(learn.predict(df.iloc[i])[1]) for i in range(1579, 1879)]

In [56]:
y_hat = np.array(y_hat)

In [57]:
loss(y_hat, y)

-0.5333333333333333

In [58]:
tn, fp, fn, tp = metrics.confusion_matrix(y, y_hat).ravel()

In [59]:
tp

14

In [60]:
fn

9

In [61]:
fp

1

In [62]:
tn

276

### Accuracy is not bad imo...

In [63]:
acc = (tp+tn)/(tp+fn+fp+tn)
acc

0.9666666666666667

## Evaluate on test

In [64]:
X_test = pd.read_csv('/home/sharif/Data-Mining-Cup/test.csv', sep="|")
y_test = pd.read_csv('/home/sharif/Data-Mining-Cup/DMC-2019-realclass.csv', sep="|")

In [65]:
len(test)

498121

In [66]:
test.head()

,trustLevel,totalScanTimeInSeconds,grandTotal,lineItemVoids,scansWithoutRegistration,quantityModifications,scannedLineItemsPerSecond,valuePerSecond,lineItemVoidsPerPosition
0,4,467,88.48,4,8,4,0.014989,0.189465,0.571429
1,3,1004,58.99,7,6,1,0.026892,0.058755,0.259259
2,1,162,14.00,4,5,4,0.006173,0.086420,4.000000
3,5,532,84.79,9,3,4,0.026316,0.159380,0.642857
4,5,890,42.16,4,0,0,0.021348,0.047371,0.210526


In [76]:
def predict():
    y_hat = []
    for i in range(len(X_test)):
        if i % 200 == 0: print(i/500000)
        y_hat.append(int(learn.predict(X_test.iloc[i])[1]))
    return y_hat